## Load Packages

In [1]:
include("printmat.jl")

println4Ps (generic function with 1 method)

In [2]:
using Plots
backend = "gr"              #"gr" (default), "pyplot" 

if backend == "pyplot"
    pyplot(size=(600,400))
else    
    gr(size=(600,400))
end

Plots.GRBackend()

# From Chapter on Mean-Variance Analysis

In [3]:
function MVTangencyP(μ,Σ,Rf)           #calculates the tangency portfolio
    n     = length(μ)
    oneV  = ones(n)  
    μe    = μ - Rf                    #excess returns            
    Σ_1   = inv(Σ)
    w     = Σ_1 *μe/(oneV'Σ_1*μe)
    muT   = w'μ + (1-sum(w))*Rf
    StdRT = sqrt(w'Σ*w)
    return w,muT,StdRT
end

MVTangencyP (generic function with 1 method)

# Several Risky Assets and one Riskfree Asset

An investor who maximizes

$\text{E}U(R_{p})   =\text{E}R_{p}-\frac{k}{2} \text{Var}(R_{p})$,

subject to 

$R_{p} = v'R^e + R_f$

will pick the portfolio weights (on the risky assets)

$v  =\frac{1}{k}\Sigma^{-1}\mu^{e}$

The portfolio weight on the riskfree asset is $1-\mathbf{1}'v$

In [4]:
μ = [0.09; 0.06]                     #means
Σ = [ 256  0;
      0    144]/100^2
Rf = 0.01
println("expected returns: ")
printmat(μ)
println("covariance matrix:")
printmat(Σ)

expected returns: 
     0.090
     0.060

covariance matrix:
     0.026     0.000
     0.000     0.014



In [5]:
function OptimalPortfolio(μ,Σ,Rf,k)           #calculates optimal portfolio weights etc
    n   = length(μ)
    μe  = μ - Rf                              #expected excess returns            
    Σ_1 = inv(Σ)
    v   = Σ_1 * μe/k                          #optimal weights risky assets, 1-1'v in riskfree   
    mup   = v'μ + (1-sum(v))*Rf               #expected return and std of optimal portfolio
    StdRp = sqrt(v'Σ*v)
    return v, mup, StdRp
end    

OptimalPortfolio (generic function with 1 method)

In [6]:
k = 15
(vOpt,muOpt,StdOpt) = OptimalPortfolio(μ,Σ,Rf,k)         #find optimal portfolio

println("optimal portfolio weights on risky and riskfree when k = $k: ")
printmat([vOpt;(1-sum(vOpt))])
println("optimal portfolio weights on risky/their sum: ")
printmat(vOpt/sum(vOpt))

(wT,muT,StdRT) = MVTangencyP(μ,Σ,Rf)
println("Tangency portfolio: ")
printmat(wT)

println("ERp and Std(Rp) for optimal portfolio when k = $k, %: ")
printmat([muOpt StdOpt]*100)

optimal portfolio weights on risky and riskfree when k = 15: 
     0.208
     0.231
     0.560

optimal portfolio weights on risky/their sum: 
     0.474
     0.526

Tangency portfolio: 
     0.474
     0.526

ERp and Std(Rp) for optimal portfolio when k = 15, %: 
     3.824     4.339



In [8]:
k = 5:1:25
K = length(k)
ERp   = fill(NaN,K)
StdRp = fill(NaN,K)
for i = 1:K
    (v_i,ER_i,Std_i) = OptimalPortfolio(μ,Σ,Rf,k[i])
    ERp[i]   = ER_i 
    StdRp[i] = Std_i
end    

println("k ERp StdRp, %:")
printmat([collect(k) ERp*100 StdRp*100])

k ERp StdRp, %:
     5.000     9.472    13.017
     6.000     8.060    10.848
     7.000     7.052     9.298
     8.000     6.295     8.136
     9.000     5.707     7.232
    10.000     5.236     6.509
    11.000     4.851     5.917
    12.000     4.530     5.424
    13.000     4.259     5.007
    14.000     4.026     4.649
    15.000     3.824     4.339
    16.000     3.648     4.068
    17.000     3.492     3.829
    18.000     3.353     3.616
    19.000     3.230     3.426
    20.000     3.118     3.254
    21.000     3.017     3.099
    22.000     2.926     2.958
    23.000     2.842     2.830
    24.000     2.765     2.712
    25.000     2.694     2.603



In [9]:
scatter(StdRp*100,ERp*100,xlims=(0,15),ylims=(0,15),legend=false)
title!("Mean vs standard deviation of optimal portfolios")
xlabel!("Std(Rp), %")
ylabel!("ERp, %")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 5 
 
 
 10 
 
 
 15 
 
 
 0 
 
 
 5 
 
 
 10 
 
 
 15 
 
 
 Mean vs standard deviation of optimal portfolios 
 
 
 Std(Rp), % 
 
 
 ERp, %